In [1]:

import os
#chat = open("../_chat.txt", encoding="utf-8")
with open("../_chat.txt",  encoding="utf-8") as f:
    chat = [line.strip('\n') for line in f]

## Clean Data

In [2]:
import re
joinedChat=' '.join(chat)
print(joinedChat[:500])

emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
joinedChat=(emoj.sub(r'', joinedChat)) # no emoji

joinedChat=re.sub(r"\[[^\]]*\]", '\n', joinedChat)
joinedChat=re.sub(r" Elif Yavrum:", 'Elif Yavrum:', joinedChat)
joinedChat=re.sub(r" Dogu Tamgac:", 'Dogu Tamgac:', joinedChat)
joinedChat=joinedChat.replace("\u200e", "")
print(joinedChat[:500])

[04.10.17, 17:32:35] Elif Yavrum: ‎Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. [04.10.17, 17:32:35] Elif Yavrum: Numaram degisti ekle beni bro [04.10.17, 18:14:23] Dogu Tamgac: Ekledim bro [05.10.17, 12:14:44] Elif Yavrum: Kk [11.10.17, 21:24:19] Dogu Tamgac: Telefon aldim ha [11.10.17, 21:24:21] Dogu Tamgac: Ekle [11.10.17, 22:04:03] Elif Yavrum: Tamam eklerim ha [11.10.17, 22:12:36] Dogu Tamgac: Napan [11.10.17, 22:13

Elif Yavrum: Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. 
Elif Yavrum: Numaram degisti ekle beni bro 
Dogu Tamgac: Ekledim bro 
Elif Yavrum: Kk 
Dogu Tamgac: Telefon aldim ha 
Dogu Tamgac: Ekle 
Elif Yavrum: Tamam eklerim ha 
Dogu Tamgac: Napan 
Elif Yavrum: Selinlerle viski iciyoruz 
Elif Yavrum: Sen 
Dogu Tamgac: Ben de kayralardayim 
Dogu Tamgac: Takiliyoz 
Dogu Tamgac: Dersler basladi mi 
Elif Yavrum: Yok 
Elif Ya

In [3]:
chars=sorted(list(set(joinedChat)))

In [4]:
vocab_size=len(set(chars))
print(''.join(chars))
print(vocab_size)

	
 !"#%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ]^_`abcdefghijklmnopqrstuvwxyz|}~ £¨«®¯´·»ÇÖ×ÜßâäçéêöüğİıŞş̈‐—’“”„•…€₺⃣
131


# Building character level language model. <br>
stoi: Map strings to integers. <br>
Then apply it and create encoding and decoding lambda functions <br>
(Normally you can use sentencepiece from google or tiktoken)

In [5]:
stoi={ch:i for i,ch in enumerate(chars)}
itoi={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stoi[i] for i in s]
decode=lambda l: ''.join([itoi[i] for i in l])
print(encode("l l"))
print(decode(encode("l l")))

[75, 2, 75]
l l


In [7]:
import torch
torch.cuda.is_available()
data=torch.tensor(encode(joinedChat), dtype=torch.long)
print(data.shape , data.dtype)
print(data[:100])

torch.Size([3742194]) torch.int64
tensor([ 1, 38, 75, 72, 69,  2, 58, 64, 85, 81, 84, 76, 27,  2, 46, 68, 82, 82,
        64, 70, 68, 82,  2, 64, 77, 67,  2, 66, 64, 75, 75, 82,  2, 64, 81, 68,
         2, 68, 77, 67, 14, 83, 78, 14, 68, 77, 67,  2, 68, 77, 66, 81, 88, 79,
        83, 68, 67, 15,  2, 47, 78,  2, 78, 77, 68,  2, 78, 84, 83, 82, 72, 67,
        68,  2, 78, 69,  2, 83, 71, 72, 82,  2, 66, 71, 64, 83, 13,  2, 77, 78,
        83,  2, 68, 85, 68, 77,  2, 56, 71, 64])


In [8]:
n = int(0.95 * len(data))
train_data=data[:n]
val_data=data[n:]

Now take the data asa blocks. We chose the block size 8 and initialized the length of the training data with 9 as there are 8 positions to guess

In [14]:
block_size= 8
batch_size= 4

train_data[:block_size+1]

tensor([ 1, 38, 75, 72, 69,  2, 58, 64, 85])

In [15]:
torch.manual_seed(1337) # to be able to reproduce
def get_batch(split):
    data=train_data if split=='train' else val_data
    #shape batchsize row vector
    ix=torch.randint(len(data) - block_size, (batch_size,))
    #stacked multiple sentences as row vectors
    x=torch.stack([data[i:i+block_size] for i in ix])
    #to train
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
xb,yb= get_batch('train')
print("input:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)



input:
torch.Size([4, 8])
tensor([[38, 75, 72, 69,  2, 58, 64, 85],
        [58, 64, 85, 81, 84, 76, 27,  2],
        [81, 67, 64, 81, 75, 64, 88, 67],
        [74, 82, 64,  2,  1, 38, 75, 72]])
targets:
torch.Size([4, 8])
tensor([[75, 72, 69,  2, 58, 64, 85, 81],
        [64, 85, 81, 84, 76, 27,  2, 34],
        [67, 64, 81, 75, 64, 88, 67, 72],
        [82, 64,  2,  1, 38, 75, 72, 69]])


import ntlk
AI_tokens=word_tokenize(joinedChat)
print(AI_tokens[:500])

from nltk.probability import FreqDist
fdist=FreqDist()
for word in AI_tokens:
    fdist[word.lower()]+=1
fdist